In [105]:
import numpy as np

class Block():
    def __init__(self, WorkingVol, K, Percent_DMSO, Compound):
        self.WorkingVol = WorkingVol *1000 # convert to nl
        self.K = K
        self.Percent_DMSO = Percent_DMSO/100 # as a fraction of 1
        self.Compound = Compound
        self.Transfer = self.MakeTransfer()
            
    def MakeTransfer(self):
        compound_vol, DMSO = self.MakeCompound_Vols()
        output = pd.DataFrame([], columns = ['Source','Dest','Vol'])
        
        for i,(j, k) in enumerate(zip(compound_vol, DMSO)):
            temp = pd.DataFrame([[self.Compound, 'A'+str(i), j], #Compound to target well
                                ['DMSO','A'+str(i), k], # DMSO to target well
                                [self.Compound, 'B'+str(i), j], # Compound to blank well
                                ['DMSO','B'+str(i), k]] , # DMSO to blank Well
                                columns = ['Source','Dest','Vol'])
            output=output.append(temp)
        output.reset_index(inplace=True,drop=True)
        
        return output
    
    def MakeCompound_Vols(self):
        compound_vol = np.linspace(0,1,8)**self.K 
        compound_vol *= self.Percent_DMSO
        compound_vol *= self.WorkingVol
        compound_vol = 2.5* np.round(compound_vol/2.5)
        DMSO = (self.Percent_DMSO * self.WorkingVol) - compound_vol
        return compound_vol, DMSO
    
    def MapWells(self,TestWellsx8, BlankWellsx8):
        TestWells = {'A' + str(i):j for i,j in zip(range(9),TestWellsx8)}
        BlankWells = {'B' + str(i):j for i,j in zip(range(9),BlankWellsx8)}
        self.Transfer['Dest'] = self.Transfer['Dest'].replace(TestWells)
        self.Transfer['Dest'] = self.Transfer['Dest'].replace(BlankWells)
        return self.Transfer
    
    def Dispense(self, compound, dmso):
        output = pd.DataFrame([], columns = ['Source','Dest','Vol'])
        for i in self.Transfer.index:
            row = self.Transfer.loc[i,:]
            source = row['Source']
            
            if source == compound.compound:
                # returns dictionary of transfers
                transfers = compound.Sample(row['Vol'])
                for j in transfers:
                    temp = pd.DataFrame([[j,row['Dest'],transfers[j]]], columns = ['Source','Dest','Vol'])
                    output = output.append(temp)
                    
            if source == dmso.compound:
                # returns dictionary of transfers
                transfers = dmso.Sample(row['Vol'])
                for j in transfers:
                    temp = pd.DataFrame([[j,row['Dest'],transfers[j]]], columns = ['Source','Dest','Vol'])
                    output = output.append(temp)
        return output.reset_index(drop=True)
            
            
block = Block(20.0,1.545455,1.,'Arachadionic acid')
block.MapWells(['C'+str(i) for i in range(1,9)],['D'+str(i) for i in range(1,9)])
block.Transfer

,Source,Dest,Vol
0,Arachadionic acid,C1,0.0
1,DMSO,C1,200.0
2,Arachadionic acid,D1,0.0
3,DMSO,D1,200.0
4,Arachadionic acid,C2,10.0
5,DMSO,C2,190.0
6,Arachadionic acid,D2,10.0
7,DMSO,D2,190.0
8,Arachadionic acid,C3,30.0
9,DMSO,C3,170.0


In [106]:
class Compound():
    def __init__(self,coumpound_name,wells):
        self.compound = coumpound_name
        self.MaxWellVol = 15 * 1000 #nl
        self.MinWellVol = 2 * 1000 #nl
        
        self.wells = self.FillWells(wells)
        
    def FillWells(self,wells):
        output = {}
        for i in wells:
            output[i] = self.MaxWellVol #ul
        return output
    
    def AvailableVolume(self):
        return sum(self.wells.values())
    
    def Sample(self,vol):
        sample = {}
        for well in self.wells:
            if self.wells[well] > self.MinWellVol:
                if vol < self.wells[well]:
                    self.wells[well] -= vol
                    sample[well] = vol
                    vol -=vol
                    break
                else:
                    sampleVol = self.wells[well]-self.MinWellVol
                    sample[well] = sampleVol
                    self.wells[well] -= sampleVol
                    vol -= sampleVol
        if vol !=0:
            print('Vol not reached')
        return sample
    
dmso = Compound('DMSO',['A1','A2','A3'])  
Arachadionic = Compound('Arachadionic acid',['B1','B2','B3'])  

In [107]:
block.Dispense(Arachadionic, dmso)

,Source,Dest,Vol
0,B1,C1,0.0
1,A1,C1,200.0
2,B1,D1,0.0
3,A1,D1,200.0
4,B1,C2,10.0
5,A1,C2,190.0
6,B1,D2,10.0
7,A1,D2,190.0
8,B1,C3,30.0
9,A1,C3,170.0


In [112]:
block = Block(20.0,4,1.,'Arachadionic acid')

block.MakeCompound_Vols()

(array([  0. ,   0. ,   2.5,   7.5,  22.5,  52.5, 107.5, 200. ]),
 array([200. , 200. , 197.5, 192.5, 177.5, 147.5,  92.5,   0. ]))